In [1]:
import os
from pathlib import Path

import syft_rds as sy
from syft_rds.orchestra import setup_rds_server
import syft_flwr

## Step 1: Setup RDS Server

In [2]:
ds_stack = setup_rds_server(email="ds@openmined.org", key="flwr")

2025-04-01 07:30:10.085 | INFO     | syft_rds.orchestra:setup_rds_server:179 - Launching mock RDS server in /var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr...
/Users/rasswanths/openmined/flwr/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:896: PydanticDeprecatedSince20: `__get_validators__` is deprecated and will be removed, use `__get_pydantic_core_schema__` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [3]:
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"
DS  = "ds@openmined.org"

In [4]:
do_client_1 = ds_stack.init_session(host=DO1)
assert do_client_1.host == DO1
assert do_client_1.email == DS
assert not do_client_1.is_admin

do_client_2 = ds_stack.init_session(host=DO2)
assert do_client_2.host == DO2
assert do_client_2.email == DS
assert not do_client_2.is_admin

datasites = [do_client_1, do_client_2]

## Step 2: Explore Datasets

In [5]:
EXAMPLE = "quickstart-pytorch"
SYFTBOX_DATASET_NAME = "cifar10"

PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE

In [6]:
dataset1 = do_client_1.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset1.describe()

/Users/rasswanths/openmined/flwr/.venv/lib/python3.12/site-packages/syft_rds/models/base.py:51: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field in self.model_fields.keys():


/Users/rasswanths/openmined/flwr/.venv/lib/python3.12/site-packages/syft_rds/utils/resources.py:24: DeprecationWarning: read_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  return importlib.resources.read_text(module, fname)
/Users/rasswanths/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/importlib/resources/_legacy.py:79: DeprecationWarning: open_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with open_text(package, resource, encoding, errors) as fp:


uid,8f491bff-df03-4bd7-9091-9f2765ecd848
created_at,2025-04-01 01:59:43
updated_at,2025-04-01 01:59:43
name,cifar10
readme_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do1@openmined.org/public/datasets/cifar10/README.md
mock_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do1@openmined.org/public/datasets/cifar10
private_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do1@openmined.org/private/datasets/cifar10


In [7]:
dataset2 = do_client_2.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset2.describe()

uid,3c81dee7-661a-474f-8624-8c1a81b3944d
created_at,2025-04-01 01:59:58
updated_at,2025-04-01 01:59:58
name,cifar10
readme_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do2@openmined.org/public/datasets/cifar10/README.md
mock_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do2@openmined.org/public/datasets/cifar10
private_path,/private/var/folders/4w/9cvj7hqd4_386stwby6n2pcw0000gn/T/flwr/datasites/do2@openmined.org/private/datasets/cifar10


## Step 3: Bootstrap Project

In [8]:
syft_flwr.bootstrap(FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2])

## Step 4: DS Submits Jobs

In [9]:
# Job submission

for client in datasites:
    job = client.jobs.submit(
        name="Syft Flower Pytorch Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated", "learning", "syft", "flwr"],
        entrypoint="main.py",
    )
    print(job)

Job
  uid: a1788396-591f-4283-94cb-930cfc068114
  created_by: ds@openmined.org
  created_at: 2025-04-01T02:00:15.826658Z
  updated_at: 2025-04-01T02:00:15.826664Z
  client_id: d526bdb8-95ed-4584-8d06-26a8dcc96569
  name: Syft Flower Pytorch Experiment
  description: Syft Flower Federated Learning Experiment
  user_code_id: 4d7babcb-98c6-4a7c-9487-cfd0b96ce02a
  tags: ['federated', 'learning', 'syft', 'flwr']
  user_metadata: {}
  status: pending_code_review
  error: no_error
  error_message: None
  output_url: syft://do1@openmined.org/app_data/RDS/user_files/ds@openmined.org/Job/a1788396-591f-4283-94cb-930cfc068114
  dataset_name: cifar10
Job
  uid: 950c9c75-e2d3-417c-94e0-c52a51659f5a
  created_by: ds@openmined.org
  created_at: 2025-04-01T02:00:16.383965Z
  updated_at: 2025-04-01T02:00:16.383972Z
  client_id: b403db0c-7646-4afd-b233-7ad6ed728ffd
  name: Syft Flower Pytorch Experiment
  description: Syft Flower Federated Learning Experiment
  user_code_id: f7a6c95b-928c-44d7-8fe6-ca99

## Run simulation using the `main.py` file directly

In [11]:
os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds_stack.client.config_path)
!uv run {str(FLWR_PROJECT_PATH / "main.py")} --active

/Users/rasswanths/openmined/flwr/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:896: PydanticDeprecatedSince20: `__get_validators__` is deprecated and will be removed, use `__get_pydantic_core_schema__` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
2025-04-01 07:34:09.364 | INFO     | syft_flwr.flower_server:syftbox_flwr_server:16 - Started SyftBox Flower Server on: ds@openmined.org
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
2025-04-01 07:34:09.367 | INFO     | syft_flwr.strategy.fedavg:_save_global_model:28 - Checkpoint saved to: /Users/rasswanths/openmined/flwr/examples/quickstart-pytorch/pytorch/weights/parameters_round_0.safetensors
INFO :      Evaluation returned no